In [1]:
import matplotlib.pyplot as plt 
import numpy as np 
import pandas
import plotly.express as px
from windrose import WindroseAxes

import xlwings as xw

/var/folders/6f/q6rj66q95cj2gdjclzvg_66m0000gp/T/ipykernel_15572/3157486580.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas


In [2]:
def intoExcel(sheetname: str, location: str, data):
    wb = xw.Book("Book1_WIND.xlsx")
    P90_sheet = wb.sheets[sheetname]
    P90_sheet.range(location).options(index=False, header = True).value = data

In [3]:
site_list = ['Albemarle', 'Boulmer', 'Charterhall', 'Gogarbank', 'Leuchars', 'Redesdale']

# 'Penmanshiel'

df_P_2016 = pandas.read_excel(f'./datasets/Penmanshiel.xlsx' , sheet_name=0)
df_P_2017 = pandas.read_excel(f'./datasets/Penmanshiel.xlsx' , sheet_name=1)


df_A_2016 = pandas.read_excel(f'./datasets/{site_list[0]}.xlsx' , sheet_name=9)
df_A_2017 = pandas.read_excel(f'./datasets/{site_list[0]}.xlsx' , sheet_name=10)

df_B_2016 = pandas.read_excel(f'./datasets/{site_list[1]}.xlsx' , sheet_name=4)
df_B_2017 = pandas.read_excel(f'./datasets/{site_list[1]}.xlsx' , sheet_name=5)

df_C_2016 = pandas.read_excel(f'./datasets/{site_list[2]}.xlsx' , sheet_name=3)
df_C_2017 = pandas.read_excel(f'./datasets/{site_list[2]}.xlsx' , sheet_name=4)

df_G_2016 = pandas.read_excel(f'./datasets/{site_list[3]}.xlsx' , sheet_name=3)
df_G_2017 = pandas.read_excel(f'./datasets/{site_list[3]}.xlsx' , sheet_name=4)

df_L_2016 = pandas.read_excel(f'./datasets/{site_list[4]}.xlsx' , sheet_name=3)
df_L_2017 = pandas.read_excel(f'./datasets/{site_list[4]}.xlsx' , sheet_name=4)

df_R_2016 = pandas.read_excel(f'./datasets/{site_list[5]}.xlsx' , sheet_name=3)
df_R_2017 = pandas.read_excel(f'./datasets/{site_list[5]}.xlsx' , sheet_name=4)

In [4]:
df_P_2017.rename(columns = {"# Date and time": 'ob_end_time'}, inplace = True)

In [5]:
# standardising column names 

list_df = [df_A_2016, df_A_2017,  df_B_2016, df_B_2017, df_C_2016, df_C_2017, df_G_2016, df_G_2017, df_L_2016, df_L_2017, df_R_2016, df_R_2017]
list_df_2017 = [df_A_2017, df_B_2017, df_C_2017, df_G_2017, df_L_2017, df_R_2017]

# P has different columns

for df in list_df:
    df.columns = ['datetime', 'mean_wind_dir', 'mean_speed_knots']


df_P_2016.columns = ['datetime', 'mean_speed_ms', 'speed_std', 'mean_wind_dir']
df_P_2017.columns = ['datetime', 'mean_speed_ms', 'speed_std', 'mean_wind_dir']

In [6]:
# removing the 'end data' row

for i in range(len(list_df)):
    list_df[i]['mean_speed_ms'] = list_df[i]['mean_speed_knots'] * (1 / 1.94384)
    
    # Drop the last row
    last_index = list_df[i].index[-1]
    list_df[i].drop(last_index, inplace=True)
    
    list_df[i]['datetime'] = pandas.to_datetime(list_df[i]['datetime'])
    list_df[i].set_index('datetime', inplace=True)




In [7]:
# resampling hourly to MCP with met stations (time only - not angles)

df_P_2016['datetime'] = pandas.to_datetime(df_P_2016['datetime'])
df_P_2017['datetime'] = pandas.to_datetime(df_P_2017['datetime'])
df_P_2016.set_index('datetime', inplace=True)
df_P_2017.set_index('datetime', inplace=True)
df_P_2016_H = df_P_2016.resample('H').mean()
df_P_2017_H = df_P_2017.resample('H').mean()

/var/folders/6f/q6rj66q95cj2gdjclzvg_66m0000gp/T/ipykernel_15572/255064806.py:7: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_P_2016_H = df_P_2016.resample('H').mean()
/var/folders/6f/q6rj66q95cj2gdjclzvg_66m0000gp/T/ipykernel_15572/255064806.py:8: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_P_2017_H = df_P_2017.resample('H').mean()


In [21]:
def get_cardinal_direction(degrees):
    if 0 <= degrees < 11.25 or 348.75 <= degrees <= 360:
        return 'N'
    elif 11.25 <= degrees < 33.75:
        return 'NNE'
    elif 33.75 <= degrees < 56.25:
        return 'NE'
    elif 56.25 <= degrees < 78.75:
        return 'ENE'
    elif 78.75 <= degrees < 101.25:
        return 'E'
    elif 101.25 <= degrees < 123.75:
        return 'ESE'
    elif 123.75 <= degrees < 146.25:
        return 'SE'
    elif 146.25 <= degrees < 168.75:
        return 'SSE'
    elif 168.75 <= degrees < 191.25:
        return 'S'
    elif 191.25 <= degrees < 213.75:
        return 'SSW'
    elif 213.75 <= degrees < 236.25:
        return 'SW'
    elif 236.25 <= degrees < 258.75:
        return 'WSW'
    elif 258.75 <= degrees < 281.25:
        return 'W'
    elif 281.25 <= degrees < 303.75:
        return 'WNW'
    elif 303.75 <= degrees < 326.25:
        return 'NW'
    elif 326.25 <= degrees < 348.75:
        return 'NNW'


In [96]:
# Merge the dataframes in such a way that the datetimes are aligned 
# X = penmanshiel 
# Y = met station 


dfs_2017_entire = []
dfs_2017_sectors = []
sections = np.arange(16)

dfs_cardinal = []

sector_summaries = []


for i in range(len(list_df_2017)):
    merged_df = pandas.merge(df_P_2017_H, list_df_2017[i], on='datetime', how='left')
    merged_df = merged_df.drop(columns = ['mean_wind_dir_x'])
    merged_df['cardinal_direction'] = merged_df['mean_wind_dir_y'].apply(get_cardinal_direction)

    merged_df = merged_df[merged_df['mean_speed_ms_x'] >= 3]

    # apply filter to Penmanshiel data but not met station as we trust met station more 

    cardinal_df = merged_df[['mean_speed_ms_x', 'mean_speed_ms_y', 'cardinal_direction']]

    merged_df = merged_df.rename(columns = {"mean_speed_ms_x" : "Mean Speed Penmanshiel", "mean_speed_ms_y" : f"Mean Speed {site_list[i]}"})

    directions = cardinal_df['cardinal_direction'].unique()

    # for direction in directions:
    #     cardinal_df[f"{direction} WindSpeed"] = cardinal_df['cardinal_direction'] == direction

    cardinal_df = cardinal_df.rename(columns = {"mean_speed_ms_x" : "Mean Speed Penmanshiel", "mean_speed_ms_y" : f"Mean Speed {site_list[i]}", "cardinal_direction" : "Cardinal Direction"})

    sector_summary_dict = {}

    sector_df = pandas.DataFrame()

    sector_df = cardinal_df[["Mean Speed Penmanshiel", f"Mean Speed {site_list[i]}", "Cardinal Direction"]]
    sector_df = sector_df.sort_values(by = "Cardinal Direction")

    # for direction in directions: 
    #     sector_df = cardinal_df[["Mean Speed Penmanshiel", f"Mean Speed {site_list[i]}", "Cardinal Direction"]][cardinal_df['Cardinal Direction'] == direction]

    # for direction in directions:
    #     # Calculate mean for each direction
        
    #     mean = cardinal_df["Mean Speed Penmanshiel"][cardinal_df[f"{direction}"]].mean()
        
    #     # Store the mean in the dictionary
    #     sector_summary_dict[f"Mean {direction} (m/s)"] = mean

    # Convert the dictionary to a DataFrame and append to sector_summaries
    # sector_summary_df = pandas.DataFrame([sector_summary_dict])
    sector_summaries.append(sector_df)


    dfs_2017_entire.append(merged_df)
    dfs_cardinal.append(cardinal_df)

    dfs_2017_sectors.append(cardinal_df)




In [90]:
dfs_2017_entire[5]

,Mean Speed Penmanshiel,speed_std,mean_wind_dir_y,mean_speed_knots,Mean Speed Redesdale,cardinal_direction
datetime,,,,,,
2017-01-01 00:00:00,8.746667,0.720000,320.0,4.0,2.057783,NW
2017-01-01 01:00:00,8.630000,0.735000,320.0,3.0,1.543337,NW
2017-01-01 02:00:00,10.081666,1.086667,340.0,7.0,3.601119,NNW
2017-01-01 03:00:00,10.353333,1.190000,340.0,5.0,2.572228,NNW
2017-01-01 04:00:00,10.585000,1.213333,360.0,9.0,4.630011,N
...,...,...,...,...,...,...
2017-12-31 19:00:00,9.543333,1.325000,230.0,12.0,6.173348,SW
2017-12-31 20:00:00,4.156667,1.255000,220.0,6.0,3.086674,SW
2017-12-31 21:00:00,5.105000,1.318333,230.0,7.0,3.601119,SW


In [87]:
dfs_cardinal[1]

,Mean Speed Penmanshiel,Mean Speed Boulmer,Cardinal Direction
datetime,,,
2017-01-01 00:00:00,8.746667,4.630011,WNW
2017-01-01 01:00:00,8.630000,3.601119,N
2017-01-01 02:00:00,10.081666,4.115565,N
2017-01-01 03:00:00,10.353333,4.630011,N
2017-01-01 04:00:00,10.585000,5.658902,N
...,...,...,...
2017-12-31 19:00:00,9.543333,6.687793,SSW
2017-12-31 20:00:00,4.156667,4.630011,SW
2017-12-31 21:00:00,5.105000,4.115565,SW


In [103]:
def intoExcel(sheetname: str, location: str, data):
    wb = xw.Book("Book1_WIND.xlsx")
    P90_sheet = wb.sheets[sheetname]
    P90_sheet.range(location).options(index=True, header = True).value = data

site_initials = ["A", "B", "C", "G", "L", "R"]

locations = ["H33", "P33", "X33", "AF33", "AN33", "AV33", "BD33", "BL33", "BT33", "CB33", "CJ33", "CR33", "CZ33", "DH33", "DP33", "DX33"]

for i in range(len(site_initials)+1):

    intoExcel(sheetname= f"P_{site_initials[i]}", location = "B33", data = dfs_cardinal[i])

    for index, direction in enumerate(directions):
        df_dir = sector_summaries[i]
        df_dir = df_dir[df_dir["Cardinal Direction"] == direction]
        df_dir.columns = [["Mean Speed Penmanshiel", f"Mean Speed {site_list[i]}", "Cardinal Direction"]]

        intoExcel(sheetname= f"P_{site_initials[i]}", location = locations[index] , data = df_dir)








IndexError: list index out of range